In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries
!pip3 install face_recognition

import torch
import torchvision
from torch import nn
from torchvision import model
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.autofrad import Variable
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition

In [ ]:
from torch import nn
from torchvision import models

class ImageModel(nn.Module):
    def __init__(self, num_classes):
        super(ImageModel, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.features = nn.Sequential(*list(model.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(2048, num_classes)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dp(self.relu(x))
        x = self.classifier(x)
        return x


In [ ]:
im_size = 224
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax()
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))

def predict(model,img,path = './'):
  fmap,logits = model(img.to('cuda'))
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  logits = sm(logits)
  _,prediction = torch.max(logits,1)
  confidence = logits[:,int(prediction.item())].item()*100
  print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)

  idx = np.argmax(logits.detach().cpu().numpy())
  bz, nc, h, w = fmap.shape
  out = np.dot(fmap[-1].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  predict = out.reshape(h,w)
  predict = predict - np.min(predict)
  predict_img = predict / np.max(predict)
  predict_img = np.uint8(255*predict_img)
  out = cv2.resize(predict_img, (im_size,im_size))
  heatmap = cv2.applyColorMap(out, cv2.COLORMAP_JET)
  img = im_convert(img[:,-1,:,:,:])
  result = heatmap * 0.5 + img*0.8*255
  cv2.imwrite('/content/1.png',result)
  result1 = heatmap * 0.5/255 + img*0.8
  r,g,b = cv2.split(result1)
  result1 = cv2.merge((r,g,b))
  plt.imshow(result1)
  plt.show()
  return [int(prediction.item()),confidence]
#img = train_data[100][0].unsqueeze(0)
#predict(model,img)

In [ ]:
# load the image name and labels from csv
class image_dataset(Dataset):
    def __init__(self,image_names,labels,transform = None):
        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = self.image_names[idx]
        parts = image_path.split('/')
        desired_parts = parts[5:]  # 5번째 이후의 부분을 가져옴
        new_file_path = '/'.join(desired_parts)
        label_info = self.labels.loc[self.labels['path'].values == new_file_path]

        label = 0  # Default label (혹시 라벨 정보가 없는 경우를 대비)
        if not label_info.empty:
            label_str = label_info['label_str'].values[0]
            if label_str == 'real':
                label = 1
            elif label_str == 'fake':
                label = 0

        # 이미지 불러오기
        image = Image.open(image_path)  # PIL 라이브러리를 사용해 이미지를 불러옴

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
#Code for making prediction
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

test_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.CenterCrop(im_size),
                                        transforms.Normalize(mean,std)])

test_image_dirs = [
    '/content/drive/MyDrive/datasets/test/real/',
    '/content/drive/MyDrive/datasets/test/fake/'
]

#Get list of all test image files
test_image_files = []  #image file들을 저장.(테스트 데이터만 저장을 한다)
for dir_path in test_image_dirs:
    test_image_files += glob.glob(os.path.join(dir_path, '*.jpg'))  # Assuming images are JPEG format

header_list = ["number","original_path","id","label","label_str","path"] #우리의 csv파일에 맞게 label list형태를 변환
label_test = pd.read_csv('/content/drive/MyDrive/datasets/labels/test.csv', names = header_list)
#필요한 부분만 가져간다.
label_test = label_test.drop(columns=["number", "original_path", "id"])

test_images = image_dataset(test_image_files,label_test,transform = test_transforms)
model = Model(2).cuda()
#model을 불러오가.
path_to_model = '/content/drive/MyDrive/datasets/checkpoint/checkpoint.pt'
model.load_state_dict(torch.load(path_to_model))
model.eval()
for i in range(0,len(test_image_files)):
  print(test_image_files[i])
  prediction = predict(model,test_images[i],'./')
  if prediction[0] == 1:
    print("REAL")
  else:
    print("FAKE")

In [ ]:
#Optional : If you want to pass full frame for prediction instead of face cropped frame
#code for full frame processing
class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        for i,frame in enumerate(self.frame_extract(video_path)):
          frames.append(self.transform(frame))
          if(len(frames) == self.count):
            break
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path)
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image